# Példa Notebook
Az OpenAI eléréséhez szükséges kulcsot a "system.json" fájlban tudod beállítani!

A szövegek vektor reprezentációvá (embedding) alakításával kereshető számsorokként "betömöríthetjük" szövegeinket. 

A tömörített szövegek eltárolhatók és keresőmotorrá alakíthatók. Ha két szöveg tartalma hasonló, általában a tömörített vektor reprezentációjuk is "közel" lesz egymáshoz. Ezzel a megoldással könnyedén hozhatunk létre szabadszöveges keresőt saját adatainkra. 

In [1]:
import sys
sys.path.insert(0, '..')  # szülőkönyvtár elérése "edih" mappa importálásához

from edih import Embeddings

In [44]:
example = Embeddings()
example.embedding("Teszt keresés")  # szöveghez tartozó vektor értékek és normalizált értékek

[-0.02496456  0.00468168  0.00762549 ... -0.00799553  0.00080988
 -0.00037004]

In [45]:
example.similarity("Volt egyszer egy Vadnyugat", "Volt egyszer egy vadnyulam")  # hasonlóság -1 és +1 közt

0.9429043927359

# Vektorműveletek
A Word2Vec aritmetika példájára: 

Király - Férfi + Nő ~= Királynő

Bővebb magyarázat: https://jalammar.github.io/illustrated-word2vec/

In [46]:
king = example.embedding("king")
prince = example.embedding("prince")
queen = example.embedding("queen")
princess = example.embedding("princess")
male = example.embedding("male")
female = example.embedding("female")

# hozzunk létre egy új értéket és ellenőrízzük
check = king - male + female
check

[-0.00082538  0.00497127  0.00820984 ...  0.00677392 -0.01348638
 -0.00521199]

In [47]:
example.find(check)  # top 10 legközelebbi szöveg (második helyen a queen)

['king',
 'queen',
 'princess',
 'female',
 'prince',
 'Volt egyszer egy vadnyulam',
 'Volt egyszer egy Vadnyugat',
 'Teszt keresés',
 'male']

In [48]:
example.similarity(check, queen)

0.9014563382377371

# Szomszédok kereső

1) A televíziós részek leírását kereshető vektorokká alakítjuk
2) Az epizódokkal kapcsolatos kérdésekből a ChatGPT segítségével keresőszavakat hozunk létre
3) A keresőszavak vektor reprezentációja és az epizódok vektorreprezentációi közül kiválasztjuk a leghasonlóbbat

In [20]:
# wikipedia alapján: https://hu.wikipedia.org/wiki/Szomsz%C3%A9dok_(els%C5%91_%C3%A9vad)
szomszedok = {
    "s01e01": "1987-et írunk. Ez a sorozat első része, melyben megismerkedünk a családokkal. Vágásiéknak kiutalják a lakást. Böhm, a közös képviselő fogadja őket. A ház tetszik nekik, bár Jutkának a környékkel vannak kifogásai. A lakásajtóban találkoznak Mégenheim Ádámmal, egy másik család családfőjével. Jutka édesapja Szőllősy Pál, aki már nem tartja a kapcsolatot lányával – mivel Jutka Vágási Ferivel összeházasodott-, nézi a Híradót, melyben a Szomszédokat reklámozzák. Itt mondják el, hogy kéthetente csütörtökön fogják leadni a részeket. Janka nénitől, Jutka nagynénijétől elhoznak egy régebbi bútort, ami akár hetvenezer forintot is megér (ekkor Jutkáék jövedelme havi kilencezer forint volt). A Takács család háromfős, Taki bácsi a rádiósokat autóztatja, Lenke néni boltos. Unokájuk Niski Alma, akinek a szülei később meghalnak autóbalesetben. Kertes házban éltek, Lenke néninek nagyon fáj, hogy el kell költözniük innen, mivel itt épül az M0-s körgyűrű. Alma udvarlója Sudár László, később a Presszó tulajdonosa, főnöke. Lenke néni mindent kipakol a lakásból, és szinte mindent el szeretne vinni, azonban a kutyát nem vihetik magukkal, mivel nem akar kutyaólat csinálni egy gyönyörű, összkomfortos lakásból. Bemutatkozik a Mágenheim család is. Hárman vannak, Juli kozmetikus, Ádám mentőorvos, Julcsi iskolás. Nagyon szomorú, hogy ott kellett hagyni a régi iskoláját. Ádámot egy sérülthöz hívják. A sérült férfi felesége felkeresi őt a lakásán, és egy borítékot szeretne átadni neki. Ádám nem fogadja el, mondván, ő csak a munkáját végezte. Mindhárman leülnek vacsorázni, bekapcsolják a rádiót, amiből éppen mese és a sorozat zenéje szól.",
    "s01e02": "Vágási Judit egy éhes kislányt karol fel. Takácsék levelet kapnak NDK-ból. Magenheimékhez felszerelik a telefont, Ádám 500 Ft borravalót ad a szakiknak. Alma idegenvezet Volánuszon, főnöke Sípos Dezső (Deziré) megfeddi Almát késése miatt, de randira hívja.",
    "s01e03": "Mivel csak Mágenéknek van telefonjuk a házban, mindenki hozzájuk jár telefonálni, melynek Juli nem tud igazán örülni. Ezért kihelyezik a telefont a lépcsőházba, ami kétforintos becsületkasszával működik. Takács Pista bácsit régi munkahelyére hívják, besegíteni. Még portásnak is beáll, de egy lefoglalt boros demizson miatt bajba kerül. Jutka terhes.",
    "s01e04": "Takácsék rokonlátogatásra Ózdra utaznak, Magenheimék telefonját ellopják, mivel éjszakára kint felejtik a folyosón. Vágásiéknál felmerül, hogy Jutkának új iskolába kell kerülnie. Az egyik Lantos utcai szomszéd, Vadászné gyakran részeg, ezért a férje az ablakon keresztül kidobja a ruháit. Feri leszokik a cigiről Jutka terhessége miatt.",
    "s01e05": "Ádámot öngyilkossági kísérlethez hívják, gyógyszerezés miatt gyomormosást rendel el. Alma kirúgja Lacit.",
    "s01e06": "Átalakul Juli szövetkezete, emiatt megemelkednek a dolgozók terhei. Julcsi nyári táborba megy, ahol Ádám távcsővel meglesi.",
    "s01e07": "Jutka iskolát vált. Két nyaraló evez a Dunán, egyikük felhevült testtel beugrik a vízbe, Ádám érkezik a mentőn. Ekkor ismeri meg Nagy Évát (Simorjay Emese), akivel később kollégák lesznek. Kiderül Feriék havi 2500-at törlesztenek Jankának, miközben 9000 Ft a bevételük. Deziré elmegy vacsorázni Takácsékhoz, ahol Lenke kellemetlen kérdésekkel zaklatja a szerelmes lovagot. Sümeghyhez betörnek, és ellopják a gyűrűjét.",
    "s01e08": "Feriék kapnak beutalót Balatonra. Taki az énekkar számára alapítványt szervez. Taki munkahelyén defektet kap, mivel az egyik kolléga kicserélte a gumikat. Juli kozmetikai versenyen vesz részt, ahol felbukkan Gábor Gábor fotósként.",
    "s01e09": "Etus autót venne Góliáttól, akiről kiderül, hogy első osztályú futballista volt. Gábor Gábor megpróbálja Julit rábeszélni a vállalkozásra. Feriék gyümölcsöt szednek, Jutka túl nagy súlyt emel, emiatt rosszul lesz. Alma anyja megérkezik NDK-ból.",
    "s01e10": "Sümeghy ajándékba ad Böhmnek egy dedikált bakelitet. Taki megmenti a dalárdát, Zsuzsa meglátogatja Jutkát. Egy kisfiú petárdakészítés közben berobbantja a pajtát. Ádám már Évikével érkezik a helyszínre.",
    "s01e11": "Julcsi továbbra is nagyon szétszórt az iskolában, rossz jegyeket kap és nem figyel órán. Végre megnyílik Gábor Gábor új szalonja, eközben Ferit és Taki bácsit az új adórendszer foglalkoztatja, de egyikük sem érti. Etus végre talál egy eladó Trabantot 43000 forintért, ami árban is megfelel, így a Magenheim családba végre kerül egy autó. Juli a szalon nyitóbuliján jól bepezsgőzik, ezzel magára haragítva a teljes családot. Vizy András civil rendőrnek próbálta eladni Sümeghy ékszereit a Keleti Pályaudvarnál, de lekapcsolják. Lenke néni szerencsésen átesik a szemműtéten, s kijöhet a kórházból. Sümeghy beszereltet riasztót, Jutka apja végtörleszt Jankánál. Feri kap egy háromhetes behívót a Honvédségbe.",
    "s01e12": "Magenheimék rendszeresen lerobbanó új Trabantja ébreszti reggelente a lakótelepet. Feri néhány hétre katonának áll, Lenke néni pedig köpenyt ragad és beugrik egy beteg eladónő helyére a közértben. Bandi úgy meg van elégedve a munkájával, hogy Bandi visszahívja dolgozni. Magenheim doktor egy csinos betegével kerül közelebbi kapcsolatba, de Julinak is vaj kerül a füle mögé, Gábor Gábor ugyanis tovább ostromolja szíve egyetlen királynőjét. Julcsi kiönti szívét Etusnak: arra panaszkodik, hogy szülei állandóan veszekednek, ráadásul angolul, hogy ő ne is értse. Desiré megpróbálja felmelegíteni kapcsolatukat, de csúnyán mellényúl, amikor lenácizza Alma német vendégeit. A lány, aki félig német, vérig sértődik és felmond.",
    "s01e13": "Jutka konfliktusba kerül az egyik szülővel az iskolában. Takácsék egy csőtörés miatt beáznak. Magenheiméknél igazi dráma zajlik, egy fényes parti után összevitatkoznak, ráadásul a doktor egyik betege meghal.",
    "s01e14": "Alma Juli szalonjában találkozik egy vezető stewardess-szel, aki állást ajánl neki a Malévnél. Vágásiékhoz ellátogatnak a Szöllősi szülők egy kínos vacsorára, amely után Jutka és Feri összevesznek. Takácsékhoz elmegy a biztosító embere, felmérni a károkat. Taki bácsi maszek munkahelyén rendőrségi úgy lesz egy külföldi ügyfél által fizetett 500 márkás bankjegyből. Mágenheimék házassága egyre mélyebb válságba süllyed.",
    "s01e15": "Julcsi és túlsúlyos barátja, Kozma Jocó unatkoznak, így szeméttakarításra jelentkeznek közös Bőhmnél. Jutka elkezd nyomozni Feri szülei után. Magenheim Julinak elege lesz a vitákból, Etushoz költözik. Magenheim újra találkozik Ágnessel, aki megpróbálja elcsábítani a doktort. A hamis márka ügyében folyik a nyomozás, Taki bácsi felmond a szerelőműhelynél. Almát felveszik a MALÉV-hez.",
    "s01e16": "Taki bácsi Vágási Ferivel együtt Mágenheimék Trabantját vezetik, amikor egy másik autós elüt egy gyalogost. Az emberek Taki bácsit hibáztatják a történtekért. Lenke nénit csalásba akarják belerángatni a bolti kollégái, de ő nem áll kötélnek. Vágási Ferit főnökei rá akarják venni, hogy ő közölje idős kollégájával a felmondását, de ő visszautasítja a kérést. Jutka iskolájában megtartják a Mikulás-ünnepséget, Julcsi is beöltözik Mikulásnak. Juli Etus lakásában fogadja Gábor Gábort, aki ott tölti az éjszakát, bár nem történik köztük semmi. A reggel váratlanul hazaérkező Etus kidobja Julit. Almát feleségül kéri Laci, de a lány nemet mond.",
    "s01e17": "Taki bácsit és Ferit beidézik a rendőrségre a gázolási ügyben. Feri felkötött karra érkezik, mivel éjszaka megsérült a másodállásában, a vagonok kirakodásánál. Jutka tovább nyomoztat a Vöröskereszttel Feri szülei után, de eredmény nélkül. Alma stewardessként egy járat fedélzetén megismerkedik egy szimpatikus fiatalemberrel, aki elhagyja az újságját, amit Alma talál meg. Magenheiméknél Juli és Ádám is hiába járja a várost színes tévé után, de Etusnak sikerül szereznie egyet. Julcsival együtt arra használják fel, hogy megpróbálják kibékíteni vele Julit és Ádámot, akik már egy hónapja külön élnek.",
    "s01e18": "Szilveszter van. Vágásiék meghívják Jutka barátnőit és Feri kollégáit közös ünneplésre. Mágenheimék kibékülnek, és amikor Ádámot behívják ügyelni, Juli úgy dönt, vele megy. Julcsi közös ünnepséget szervez a házban lakó gyerekeknek, amihez Böhm bácsi adja a zenét. Takácsék azt tervezik, hármasban otthon ünnepelnek, de Almát munka után követi és csellel elhívja hazulról az a fiatalember, akivel munka közben az egyik repülőjáraton ismerkedett meg.",
}

In [28]:
# epizódok átalakítása vektor reprezentációvá
from edih import Embeddings, ChatGPT
import numpy as np
from tqdm import tqdm

e = Embeddings()
search = {}
for key, value in tqdm(szomszedok.items()):
    search[key] = e.embedding(value)

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:33<00:00,  1.88s/it]


In [ ]:
# keresési feltétel
query = "Melyik részben kerül Vágási Feri slamasztikába?"

In [24]:
# keresési feltétel átalakítása kereőszavakká

converter = ChatGPT("Alakítsd át az alábbi Szomszédok televíziós sorozattal kapcsolatos kérdést csak kereszőszavakká:")
keywords = converter.chat(query)
print(keywords)

Vágási Feri + Szomszédok + slamasztika + epizód


In [ ]:
# keresőszavak átalakítása vektor reprezentációvá
keywords_as_embedding = e.embedding(keywords)

In [33]:
# legmegfelelőbb epizód megtlaálása
best_episode, best_score = "", -np.inf
for key, value in search.items():
    score = e.similarity(value, keywords_as_embedding)  # cosine hasonlóság
    if score > best_score:
        best_episode = key
        best_score = score

print(f"{best_episode} ({best_score*100.:.2f}%)\n{search[best_episode]}")

s01e16 (86.89%)
Taki bácsi Vágási Ferivel együtt Mágenheimék Trabantját vezetik, amikor egy másik autós elüt egy gyalogost. Az emberek Taki bácsit hibáztatják a történtekért. Lenke nénit csalásba akarják belerángatni a bolti kollégái, de ő nem áll kötélnek. Vágási Ferit főnökei rá akarják venni, hogy ő közölje idős kollégájával a felmondását, de ő visszautasítja a kérést. Jutka iskolájában megtartják a Mikulás-ünnepséget, Julcsi is beöltözik Mikulásnak. Juli Etus lakásában fogadja Gábor Gábort, aki ott tölti az éjszakát, bár nem történik köztük semmi. A reggel váratlanul hazaérkező Etus kidobja Julit. Almát feleségül kéri Laci, de a lány nemet mond.
